## Import libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,precision_recall_fscore_support
from sklearn.metrics import f1_score,roc_auc_score
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from xgboost import plot_importance

## Anomaly-based IDS

### Generate the datasets for unknown attack detection

In [3]:
df=pd.read_csv('./CAN_INTRUSION_sample_km.csv')

In [4]:
df

,id,d0,d1,d2,d3,d4,d5,d6,d7,Label
0,0.040771,-0.385759,-0.833987,-0.561275,-0.761493,-0.637519,-0.741096,-0.386449,-0.648399,3
1,0.040771,-0.385759,-0.833987,-0.561275,-0.761493,-0.637519,-0.741096,-0.386449,-0.648399,2
2,0.040771,-0.385759,-0.833987,-0.561275,-0.761493,-0.637519,-0.741096,-0.386449,-0.648399,1
3,0.040771,-0.385759,-0.833987,-0.561275,-0.761493,-0.637519,-0.741096,-0.386449,-0.648399,1
4,0.040771,-0.385759,-0.833987,-0.561275,-0.761493,-0.637519,-0.741096,-0.386449,-0.648399,1
...,...,...,...,...,...,...,...,...,...,...
42269,-0.949247,2.181947,1.993519,-0.561275,-0.761493,-0.609940,0.901659,-0.192963,1.197599,4
42270,-0.949247,2.181947,1.993519,-0.561275,-0.761493,-0.361730,0.901659,-0.149967,1.556887,4
42271,-0.949247,2.137485,1.993519,-0.561275,-0.761493,-0.223835,0.994396,-0.321954,1.519719,4
42272,-0.949247,1.981866,1.993519,-0.561275,-0.761493,-0.472045,0.901659,-0.149967,1.395827,4


In [5]:
df.Label.value_counts()

,count
Label,
4,29667
2,4046
3,3918
0,3187
1,1456


In [6]:
df1 = df[df['Label'] != 3]
df1['Label'][df1['Label'] < 4] = 1
df1['Label'][df1['Label'] == 4 ] = 0
df1.to_csv('./can_intrusion_sample_km_without_x_attack.csv',index=0)

In [7]:
df2 = df[df['Label'] == 3]
df2['Label'][df2['Label'] == 3] = 1
df2.to_csv('./can_intrusion_sample_km_with_x_attack.csv',index=0)

### Read the generated datasets for unknown attack detection

In [8]:
df1 = pd.read_csv('./can_intrusion_sample_km_without_x_attack.csv')
df2 = pd.read_csv('./can_intrusion_sample_km_with_x_attack.csv')

In [9]:
features = df1.drop(['Label'],axis=1).dtypes[df1.dtypes != 'object'].index
df1[features] = df1[features].apply(
    lambda x: (x - x.mean()) / (x.std()))
df2[features] = df2[features].apply(
    lambda x: (x - x.mean()) / (x.std()))
df1.replace([np.inf, -np.inf], np.nan, inplace=True)
df2.replace([np.inf, -np.inf], np.nan, inplace=True)
df1 = df1.fillna(0)
df2 = df2.fillna(0)

In [10]:
df1.Label.value_counts()

,count
Label,
0,29667
1,8689


In [11]:
df2.Label.value_counts()

,count
Label,
1,3918


In [12]:
df2p=df1[df1['Label']==0]
df2pp=df2p.sample(n=None, frac=3918/29667, replace=False, weights=None, random_state=None, axis=0)
df2=pd.concat([df2, df2pp])

In [13]:
df2.Label.value_counts()

,count
Label,
1,3918
0,3918


In [14]:
df = pd.concat([df1, df2])

In [15]:
X = df.drop(['Label'],axis=1) .values
y = df.iloc[:, -1].values.reshape(-1,1)
y=np.ravel(y)
pd.Series(y).value_counts()

,count
0,33585
1,12607


### Feature engineering (IG, FCBF, and KPCA)

#### Feature selection by information gain (IG)

In [16]:
from sklearn.feature_selection import mutual_info_classif
importances = mutual_info_classif(X, y)

In [17]:
# calculate the sum of importance scores
f_list = sorted(zip(map(lambda x: round(x, 4), importances), features), reverse=True)
Sum = 0
fs = []
for i in range(0, len(f_list)):
    Sum = Sum + f_list[i][0]
    fs.append(f_list[i][1])

In [18]:
# select the important features from top to bottom until the accumulated importance reaches 90%
f_list2 = sorted(zip(map(lambda x: round(x, 4), importances/Sum), features), reverse=True)
Sum2 = 0
fs = []
for i in range(0, len(f_list2)):
    Sum2 = Sum2 + f_list2[i][0]
    fs.append(f_list2[i][1])
    if Sum2>=0.9:
        break

In [19]:
X_fs = df[fs].values

In [20]:
X_fs.shape

(46192, 8)

In [21]:
X_fs

array([[-0.80112059, -0.60409739, -0.67566537, ..., -0.78114131,
        -0.69293059,  0.02599843],
       [-0.80112059, -0.60409739, -0.67566537, ..., -0.78114131,
        -0.69293059,  0.02599843],
       [-0.80112059, -0.60409739, -0.67566537, ..., -0.78114131,
        -0.69293059,  0.02599843],
       ...,
       [-0.80112059, -0.60409739, -0.67566537, ..., -0.78114131,
        -0.69293059,  1.73259191],
       [-0.80112059,  1.40570957, -0.66377247, ..., -0.72806909,
        -0.69293059, -0.05866978],
       [ 1.18413169, -0.60409739,  0.03790903, ...,  0.91716965,
         1.71905861, -1.03235412]])

#### Feature selection by Fast Correlation Based Filter (FCBF)

The module is imported from the GitHub repo: https://github.com/SantiagoEG/FCBF_module

In [22]:
!pip install scikit-optimizer

In [23]:
from skopt import gp_minimize
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

def fcbf_objective(threshold, X, y, clf=RandomForestClassifier(random_state=42)):
    selector = FCBF(th=threshold)
    X_selected = selector.fit_transform(X, y)
    if X_selected.shape[1] == 0:
        return 1.0
    score = cross_val_score(clf, X_selected, y, cv=3, scoring="accuracy").mean()
    return -score

In [24]:
from FCBF_module import FCBF, FCBFK, FCBFiP, get_i

res = gp_minimize(
    lambda th: fcbf_objective(th[0], X_fs, y),
    dimensions=[(0.01, 0.5)],
    n_calls=20,
    random_state=42,
    acq_func='EI',
)

best_threshold = res.x[0]
print("Best threshold:", best_threshold)
print("Best accuracy:", -res.fun)

Best threshold: 0.09988304703442026
Best accuracy: 0.7391106865471974


In [25]:
from FCBF_module import FCBF, FCBFK, FCBFiP, get_i
fcbf = FCBF(th = best_threshold)

In [26]:
X_fss = fcbf.fit_transform(X_fs,y)

In [27]:
X_fss.shape

(46192, 1)

In [28]:
X_fss

array([[-0.60409739],
       [-0.60409739],
       [-0.60409739],
       ...,
       [-0.60409739],
       [ 1.40570957],
       [-0.60409739]])

####  kernel principal component analysis (KPCA)

In [ ]:
from skopt import gp_minimize
from skopt.space import Integer, Categorical
from sklearn.decomposition import KernelPCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

def kpca_objective(params, X, y):
    n_components, kernel = params
    n_components = min(n_components, X.shape[1] - 1)
    if n_components < 1: n_components = 1
    try:
        kpca = KernelPCA(n_components=n_components, kernel=kernel, fit_inverse_transform=False, random_state=42)
        X_kpca = kpca.fit_transform(X)
        clf = RandomForestClassifier(random_state=42)
        score = cross_val_score(clf, X_kpca, y, cv=3, scoring="accuracy").mean()
        return -score
    except Exception as e:
        return 1.0

search_space = [
    Integer(2, 50),
    Categorical(['rbf', 'poly'])
]

result = gp_minimize(
    lambda params: kpca_objective(params, X_fss, y),
    search_space,
    n_calls=20,
    random_state=42,
    acq_func='EI'
)

best_n_components, best_kernel = result.x
print(f"Best n_components: {best_n_components}, Best kernel: {best_kernel}")
print(f"Best accuracy: {-result.fun:.4f}")

Best n_components: 40, Best kernel: poly
Best accuracy: 0.7390


In [29]:
from skopt import gp_minimize
from skopt.space import Integer, Categorical
from sklearn.decomposition import KernelPCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

kpca = KernelPCA(n_components=best_n_components, kernel=best_kernel, random_state=42)
X_kpca = kpca.fit_transform(X_fss)

### Train-test split after feature selection

In [30]:
X_train = X_kpca[:len(df1)]
y_train = y[:len(df1)]
X_test = X_kpca[len(df1):]
y_test = y[len(df1):]

### Apply the cluster labeling (CL) k-means method

In [31]:
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN,MeanShift
from sklearn.cluster import SpectralClustering,AgglomerativeClustering,AffinityPropagation,Birch,MiniBatchKMeans,MeanShift
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from sklearn.metrics import classification_report
from sklearn import metrics

In [32]:
def CL_kmeans(X_train, X_test, y_train, y_test,n,b=100):
    km_cluster = MiniBatchKMeans(n_clusters=n,batch_size=b)
    result = km_cluster.fit_predict(X_train)
    result2 = km_cluster.predict(X_test)

    count=0
    a=np.zeros(n)
    b=np.zeros(n)
    for v in range(0,n):
        for i in range(0,len(y_train)):
            if result[i]==v:
                if y_train[i]==1:
                    a[v]=a[v]+1
                else:
                    b[v]=b[v]+1
    list1=[]
    list2=[]
    for v in range(0,n):
        if a[v]<=b[v]:
            list1.append(v)
        else:
            list2.append(v)
    for v in range(0,len(y_test)):
        if result2[v] in list1:
            result2[v]=0
        elif result2[v] in list2:
            result2[v]=1
        else:
            print("-1")
    print(classification_report(y_test, result2))
    cm=confusion_matrix(y_test,result2)
    acc=metrics.accuracy_score(y_test,result2)
    print(str(acc))
    print(cm)

In [33]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [34]:
CL_kmeans(X_train, X_test, y_train, y_test, 8)

              precision    recall  f1-score   support

           0       0.50      1.00      0.67      3918
           1       0.00      0.00      0.00      3918

    accuracy                           0.50      7836
   macro avg       0.25      0.50      0.33      7836
weighted avg       0.25      0.50      0.33      7836

0.5
[[3918    0]
 [3918    0]]


### Hyperparameter optimization of CL-k-means
Tune "k"

In [35]:
#Hyperparameter optimization by BO-GP
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from sklearn import metrics

space  = [Integer(2, 50, name='n_clusters')]
@use_named_args(space)
def objective(**params):
    km_cluster = MiniBatchKMeans(batch_size=100, **params)
    n=params['n_clusters']

    result = km_cluster.fit_predict(X_train)
    result2 = km_cluster.predict(X_val)

    count=0
    a=np.zeros(n)
    b=np.zeros(n)
    for v in range(0,n):
        for i in range(0,len(y_train)):
            if result[i]==v:
                if y_train[i]==1:
                    a[v]=a[v]+1
                else:
                    b[v]=b[v]+1
    list1=[]
    list2=[]
    for v in range(0,n):
        if a[v]<=b[v]:
            list1.append(v)
        else:
            list2.append(v)
    for v in range(0,len(y_val)):
        if result2[v] in list1:
            result2[v]=0
        elif result2[v] in list2:
            result2[v]=1
        else:
            print("-1")
    cm=metrics.accuracy_score(y_val,result2)
    print(str(n)+" "+str(cm))
    return (1-cm)
from skopt import gp_minimize
import time
t1=time.time()
res_gp = gp_minimize(objective, space, n_calls=20, random_state=0)
t2=time.time()
print(t2-t1)
print("Best score=%.4f" % (1-res_gp.fun))
print("""Best parameters: n_clusters=%d""" % (res_gp.x[0]))

30 0.791970802919708
43 0.7924921793534933
43 0.7953597497393118
43 0.8005735140771637
32 0.791058394160584
20 0.7828467153284672
16 0.7686392075078207
5 0.7686392075078207
15 0.7734619395203337
25 0.7829770594369134
50 0.8022679874869656
50 0.8037017726798749
50 0.8012252346193952
50 0.8014859228362878
50 0.804614181438999
50 0.802007299270073
50 0.7966631908237748
50 0.7885818561001042
39 0.7907977059436914
50 0.8031803962460897
6.1931843757629395
Best score=0.8046
Best parameters: n_clusters=50


In [36]:
CL_kmeans(X_train, X_val, y_train, y_val, res_gp.x[0])

              precision    recall  f1-score   support

           0       0.81      0.97      0.88      5897
           1       0.73      0.24      0.36      1775

    accuracy                           0.80      7672
   macro avg       0.77      0.61      0.62      7672
weighted avg       0.79      0.80      0.76      7672

0.8037017726798749
[[5739  158]
 [1348  427]]


### Apply the CL-k-means model with biased classifiers

95% of the code has been shared, and the remaining 5% is retained for future extension.  
Thank you for your interest and more details are in the paper.

In [37]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    recall_score
)
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.neighbors import NearestNeighbors
import numpy as np
from skopt import BayesSearchCV
from skopt.space import Integer
from imblearn.over_sampling import SMOTE

def Anomaly_IDS(X_train, X_test, y_train, y_test, n, b=100):
    km = MiniBatchKMeans(n_clusters=n, batch_size=b, random_state=42)
    train_labels = km.fit_predict(X_train)
    test_labels  = km.predict(X_test)

    counts_pos = np.zeros(n)
    counts_neg = np.zeros(n)
    for idx, c in enumerate(train_labels):
        if y_train[idx] == 1: counts_pos[c] += 1
        else:               counts_neg[c] += 1

    cluster_prob = {}
    normal_clusters = []
    attack_clusters = []
    for c in range(n):
        tot = counts_pos[c] + counts_neg[c]
        if counts_pos[c] > counts_neg[c]:
            attack_clusters.append(c)
            cluster_prob[c] = counts_pos[c] / tot if tot>0 else 0.0
        else:
            normal_clusters.append(c)
            cluster_prob[c] = counts_neg[c] / tot if tot>0 else 0.0

    y_km = np.array([1 if c in attack_clusters else 0 for c in test_labels])

    print("CL-k-means Performance:")
    print(classification_report(y_test, y_km))
    cm = confusion_matrix(y_test, y_km)
    tn, fp, fn, tp = cm.ravel()
    dr = tp/(tp+fn) if tp+fn>0 else 0
    far = fp/(fp+tn) if fp+tn>0 else 0
    print(f"  DR: {dr:.4f}, FAR: {far:.4f}\n  CM:\n{cm}\n")

    y_train_km = np.array([1 if c in attack_clusters else 0 for c in train_labels])
    fp_idx = np.where((y_train_km == 1) & (y_train == 0))[0]
    fn_idx = np.where((y_train_km == 0) & (y_train == 1))[0]

    X_fp = X_train[fp_idx]
    X_fn = X_train[fn_idx]
    X_norm = X_train[y_train == 0]
    X_adv  = X_train[y_train == 1]

    if len(X_fp) and len(X_adv):
        attack_samples_for_fp = X_adv[np.random.choice(len(X_adv), size=len(X_fp),replace=True)]
        Xp = np.concatenate([X_fp, attack_samples_for_fp])
        yp = np.concatenate([np.zeros(len(X_fp)), np.ones(len(X_fp))])
        yp = yp.astype(np.int64)
        y_counts = np.bincount(yp)
        print(y_counts)
        min_groups = np.min(y_counts)
        if min_groups >= 2:
          opt_rfp = BayesSearchCV(
              RandomForestClassifier(random_state=0),
              {
                  'n_estimators': Integer(10,200),
                  'max_depth':    Integer(3,50),
                  'min_samples_split': Integer(2,10)
              },
              n_iter=20,
              cv=StratifiedKFold(5, shuffle=True, random_state=0),
              scoring='f1',
              n_jobs=-1,
              random_state=0
          )
          opt_rfp.fit(Xp, yp)
          rfp = opt_rfp.best_estimator_
        else:
          rfp = RandomForestClassifier(random_state=0).fit(X_train, y_train)
    else:
        rfp = RandomForestClassifier(random_state=0).fit(X_train, y_train)

    if len(X_fn) and len(X_norm):
        normal_samples_for_fn = X_norm[np.random.choice(len(X_norm), size=len(X_fn),replace=True)]
        Xn = np.concatenate([X_fn, normal_samples_for_fn])
        yn = np.concatenate([np.zeros(len(X_fn)), np.ones(len(X_fn))])
        yn = yn.astype(np.int64)
        y_counts = np.bincount(yn)
        print(y_counts)
        min_groups = np.min(y_counts)

        if min_groups >= 2:
          opt_rfn = BayesSearchCV(
              RandomForestClassifier(random_state=0),
              {
                  'n_estimators': Integer(10,200),
                  'max_depth':    Integer(3,50),
                  'min_samples_split': Integer(2,10)
              },
              n_iter=20,
              cv=StratifiedKFold(5, shuffle=True, random_state=0),
              scoring='f1',
              n_jobs=-1,
              random_state=0
          )
          opt_rfn.fit(Xn, yn)
          rfn = opt_rfn.best_estimator_
        else:
          rfn = RandomForestClassifier(random_state=0).fit(X_train, y_train)  # fallback
    else:
        rfn = RandomForestClassifier(random_state=0).fit(X_train, y_train)  # fallback

    probs = np.array([cluster_prob.get(c,0.0) for c in test_labels])
    best_thr, best_rec = 0.5, 0.0
    for thr in np.linspace(0.5, 0.99, 50):
        y_tmp = y_km.copy()
        for i, p in enumerate(probs):
            if p < thr:
                if y_tmp[i] == 0:
                    y_tmp[i] = rfn.predict(X_test[i].reshape(1,-1))[0]
                else:
                    y_tmp[i] = rfp.predict(X_test[i].reshape(1,-1))[0]
        rec = recall_score(y_test, y_tmp)
        if rec > best_rec:
            best_rec, best_thr = rec, thr

    y_final = y_km.copy()
    for i, p in enumerate(probs):
        if p < best_thr:
            if y_final[i] == 0:
                y_final[i] = rfn.predict(X_test[i].reshape(1,-1))[0]
            else:
                y_final[i] = rfp.predict(X_test[i].reshape(1,-1))[0]


    print("MTH-IDS Performance:")
    print(classification_report(y_test, y_final))
    cm2 = confusion_matrix(y_test, y_final)

    tn, fp, fn, tp = cm2.ravel()
    dr2 = tp/(tp+fn) if tp+fn>0 else 0
    far2 = fp/(fp+tn) if fp+tn>0 else 0
    acc2 = accuracy_score(y_test, y_final)

    print(f"  Acc: {acc2:.4f}, DR: {dr2:.4f}, FAR: {far2:.4f}\n  CM:\n{cm2}")

    return acc2, dr2, far2, cm2

In [38]:
X_train = X_kpca[:len(df1)]
y_train = y[:len(df1)]
X_test = X_kpca[len(df1):]
y_test = y[len(df1):]

In [39]:
one_acc, one_dr, one_far, one_f1 = Anomaly_IDS(X_train, X_test, y_train, y_test, res_gp.x[0])

CL-k-means Performance:
              precision    recall  f1-score   support

           0       0.49      0.97      0.65      3918
           1       0.19      0.01      0.02      3918

    accuracy                           0.49      7836
   macro avg       0.34      0.49      0.33      7836
weighted avg       0.34      0.49      0.33      7836

  DR: 0.0082, FAR: 0.0342
  CM:
[[3784  134]
 [3886   32]]

[862 862]
[6599 6599]
MTH-IDS Performance:
              precision    recall  f1-score   support

           0       0.73      0.92      0.81      3918
           1       0.90      0.65      0.76      3918

    accuracy                           0.79      7836
   macro avg       0.81      0.79      0.78      7836
weighted avg       0.81      0.79      0.78      7836

  Acc: 0.7888, DR: 0.6526, FAR: 0.0750
  CM:
[[3624  294]
 [1361 2557]]
